In [ ]:
from dksr.micro_mobility_MDS import * 
import matplotlib.pyplot as plt
import seaborn as sns

three datasets: trip data (df), a network model NACH (gdf), and a graph containing the bike networks corresponding data like OSMid (G); the network model NACH is based on the graph G

In [ ]:
df = pd.read_json("data.json")

In [ ]:
df = pd.json_normalize(df['sensordata'])

In [ ]:
#change in function --> "return df" in micro_mobility_MDS.py
df = clean_columns(df, inplace=True)

In [ ]:
G = ox.graph_from_bbox(50.007215, 49.996002, 8.243716, 8.275259, network_type='bike')

In [ ]:
gdf_raw = gpd.read_file('joined_graph_NACH_1500_v03.gpkg')

In [ ]:
gdf_raw.columns

In [ ]:
gdf = gdf_raw[['u', 'v','key', 'osmid', 'name', 'highway', 'maxspeed','NACH', 'geometry']]

In [ ]:
# even the NACH model is based on the G, many G.edges have NULL values for NACH
gdf.isnull().sum()

In [ ]:
# NACH seems to be relative broad distributed with std = 1.65
gdf.describe()

In [ ]:
#G.nodes are prvided as floats, which is false
gdf[['u','v','key']] = gdf[['u','v','key']].astype(np.int64)

In [ ]:
gdf['highway'].unique()

In [ ]:
sorted_data = gdf.groupby('highway')['NACH'].mean().sort_values(ascending=False)

In [ ]:
ax = sns.barplot(x=sorted_data.index, y=sorted_data.values)#, x=sorted_data['highway'], y=sorted_data['NACH'])
ax.set_xticklabels(ax.get_xticklabels(),rotation=90);

In [ ]:
sorted_data.index

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

#  NACH35 is has a radius of 1500 meters and is aligned with the average scooter movment (local perspective)
sns.scatterplot(data=gdf_raw['NACH'], ax=axes[0])
axes[0].set_title('NACH')

# NACH35 is has a radius of 3500 meters and is aligned with to overall map (global perspective)
sns.scatterplot(data=gdf_raw['NACH35'], ax=axes[1])
axes[1].set_title('NACH35')



In [ ]:
gdf.head(10)

In [ ]:
gdf['nodes'] = list(zip(gdf['u'], gdf['v'],gdf['key'] ))

In [ ]:
#a loop for getting the OSMids or other edge data that are represented in the NACH model
# this will be used later to merge scooter counts with the NACH model
osmids = []
for i,x in gdf.iterrows():
    try:
        osmids.append(G.get_edge_data(osmid = G.get_edge_data(gdf.iloc[0]['nodes'][0], gdf.iloc[0]['nodes'][1],gdf.iloc[0]['nodes'][2]).get('osmid')))
    except:
        try:
            osmids.append(G.get_edge_data(gdf.iloc[i]['u'],gdf.iloc[i]['v'], 1).get('osmid'))
        except:
            continue

In [ ]:
#a loop for getting the OSMids or other edge data that are represented in the NACH model
# this will be used later to merge scooter counts with the NACH model
osmids = []
for i,x in gdf.iterrows():
    try:
        osmids.append(G.get_edge_data(gdf.iloc[i]['nodes'][0], gdf.iloc[i]['nodes'][1],gdf.iloc[i]['nodes'][2]).get('osmid'))
    except:
        continue

In [ ]:
#check missing vallues

# the original G has about 2.7k edges
print("Original G:", len(G.edges))
#the NACH model has only about 1.7k edges
print("NACH Model:", len(gdf.index))
#lastly 1375 can mapped back on the graph, meaning some edges in the NACH model get disturbed during the process
print("Mapped Edges:", len(osmids))

In [ ]:
#find all rows that has no corresponding edge data
def missing_data(gdf,G):
    missing_edges = []
    for i, x in gdf.iterrows():
        try:
            G.get_edge_data(gdf.iloc[i]['nodes'][0], gdf.iloc[i]['nodes'][1], gdf.iloc[i]['nodes'][2]).get('osmid')
        except:
            missing_edges.append(i)

    filtered_gdf = gdf.loc[missing_edges]
    return filtered_gdf

In [ ]:
missing_data_1 = missing_data(gdf, G)

In [ ]:
missing_data_1

In [ ]:
#missing values seem to be highly relevant
missing_data_1.plot()

In [ ]:
# during export u, and v seems to be switched

def find_nodes(osmid):
    osmid = 203848419 

    # Iterate over the edges and find the matching edge
    matching_edge = None
    for u, v, key, data in G.edges(keys=True, data=True):
        if "osmid" in data and data["osmid"] == osmid:
            matching_edge = (u, v, key)
            break

    if matching_edge is not None:
        print("Matching edge found!")
        print("u:", matching_edge[0])
        print("v:", matching_edge[1])
        print("key:", matching_edge[2])
    else:
        print("No matching edge found for the given OSM ID.")

In [ ]:
find_nodes(203848419)

In [ ]:
missing_data_1[['u', 'v']] = missing_data_1[['v', 'u']]
missing_data_1['nodes'] = list(zip(missing_data_1['u'], missing_data_1['v'],missing_data_1['key'] ))

In [ ]:
edges = []
for i,x in missing_data_1.iterrows():
    try:
        edges.append(G.get_edge_data(missing_data_1.iloc[i]['nodes'][0], missing_data_1.iloc[i]['nodes'][1],missing_data_1.iloc[i]['nodes'][2]))
    except:
        continue

In [ ]:
len(edges)

In [ ]:
#drop all none types
edges = [edge for edge in edges if edge is not None]

In [ ]:
gdf_2 = gpd.GeoDataFrame.from_records(edges)

In [ ]:
gdf_2

In [ ]:
gdf_2 = gpd.GeoDataFrame(edges)

In [ ]:
gdf_2[0]

In [ ]:
df = pd.DataFrame(gdf_2[0].tolist()) 
gdf_normalized = gpd.GeoDataFrame(df)

In [ ]:
gdf = gpd.GeoDataFrame.from_records(edges)

In [ ]:
gdf_normalized

In [ ]:
G.get_edge_data(574492655, 529463, 0)

In [ ]:
edges_outview = list(G.edges)

In [ ]:
edges_outviewedges_set = set(edges_outview)

In [ ]:
gdf_edges_set = set(zip(gdf['u'], gdf['v']))

In [ ]:
missing_edges = edges_set - gdf_edges_set

In [ ]:
missing_edges = list(missing_edges)

edge_info = []
for i in missing_edges:
    try:
        edge_info.append(G.get_edge_data(missing_edges[0][0],missing_edges[0][1],missing_edges[0][2]))
    except:
        continue

In [ ]:
#edge_info

In [ ]:
df.head(2)

In [ ]:
df['drop_off_lon'] = df['coordinates'].apply(lambda x: x[-1][0])
df['drop_off_lat'] = df['coordinates'].apply(lambda x: x[-1][1])

df['start_lon'] = df['coordinates'].apply(lambda x: x[0][0])
df['start_lat'] = df['coordinates'].apply(lambda x: x[0][1])

In [ ]:
# Get the nearest nodes for start and drop-off locations
df['start_node'] = df.apply(lambda row: ox.nearest_nodes(G, row['start_lon'], row['start_lat']), axis=1)
df['dropoff_node'] = df.apply(lambda row: ox.nearest_nodes(G, row['drop_off_lon'], row['drop_off_lat']), axis=1)

In [ ]:
def get_edges(data):
    #data = np.array(data)
    edges = ox.nearest_edges(G,[k[0] for k in data], [k[1] for k in data])    
    edges = list(dict.fromkeys(edges))
    edges = [k[0:2] for k in edges]
    return edges

In [ ]:
df

In [ ]:
# Extract the routes and their lengths
routes = list(df['route'])
lengths = [nx.shortest_path_length(G, start_node, dropoff_node, weight='length') for start_node, dropoff_node in zip(df['start_node'], df['dropoff_node'])]


In [ ]:
# Step 1: Calculate betweeness centraliy for every edge and assign to string
# Alternative: replace varibale 'lenght' in G with values from betweenes centrality


len(lengths)

In [ ]:
edge_betweenness = nx.edge_betweenness_centrality(G)

In [ ]:
df.head()

In [ ]:
# G.add_weighted_edges_from

In [ ]:
df['shortestPath_length'] = pd.Series(lengths)
df['path_coeff'] = df['trip_distance']/df['shortestPath_length']

In [ ]:
df['shortestPath'] = pd.Series([[(G.nodes[node]['x'], G.nodes[node]['y']) for node in route] for route in routes])

In [ ]:
df